In [4]:
from senti_classifier import senti_classifier
from TwitterAPI import TwitterAPI
from datetime import datetime
import ConfigParser
import pickle
import nltk
import re

In [5]:
# Connect to the Twitter API and return a TwitterAPI object to use.
def get_twitter(config_file):
    config = ConfigParser.ConfigParser()
    config.read(config_file)
    twitter = TwitterAPI(
                   config.get('twitter', 'consumer_key'),
                   config.get('twitter', 'consumer_secret'),
                   config.get('twitter', 'access_token'),
                   config.get('twitter', 'access_token_secret'))
    return twitter

twitter = get_twitter('twitter.cfg')
print('Established Twitter connection.')

Established Twitter connection.


In [119]:
def process_tweet(tweet):
    #Convert to lower case
    tweet = tweet.lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','AT_USER',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
    
    return tweet


def extract_id(tweet_request_response, min_id=998816577052196870):
    """
    Extract the min of tweets' id, use a bad hack to initialize max_id
    """
    for tweet in tweet_request_response:
        if tweet['id'] < min_id:
            min_id = tweet['id']
        
    return min_id

def gather_tweets(search_query, since, until, tweets_list=[]):
    tweets = twitter.request('search/tweets',{'q':search_query, 'lang':'en', 'since':since, 'until':until, 'count':100})
    min_id = extract_id(tweets)
    
    while True:
            if not tweets.get_rest_quota():
                return tweets_list
        
            try:
                tweets = twitter.request('search/tweets',
                             {'q' : search_query, 
                              'lang' : 'en', 
                              'since' : since, 
                              'until' : until,
                              'max_id' : min_id-1,
                              'count' : 100})
            except:
                print 'No more quota !'
                return tweets_list
            
            for tweet in tweets:
                process_text = process_tweet(tweet['text'])
                tweets_list.append({'text': process_text, 
                                    'author': tweet['user']['name'], 
                                    'date': datetime.strptime(tweet['created_at'], '%a %b %d %H:%M:%S +0000 %Y').strftime('%Y-%m-%d'),
                                    'time': datetime.strptime(tweet['created_at'], '%a %b %d %H:%M:%S +0000 %Y').strftime('%H:%M:%S'),
                                   })
                
            if (extract_id(tweets, min_id=min_id) == min_id):
                return tweets_list
            else:
                min_id = extract_id(tweets, min_id=min_id)
            
            print 'Next max_id: ' + str(min_id)
            print 'Remaining quota: ' + str(tweets.get_rest_quota())
    
    return tweets_list 


In [131]:
period = ['2015-11-29', '2015-11-30']
tweets_dico_list = gather_tweets('%24GOOG', period[0], period[1])
print 'Period ' + period[0] + ' to ' + period[1] + ' gathered.'
pickle.dump(tweets_dico_list, open('save-'+period[0]+'-to-'+period[1]+'.pkl', 'wb' ))
print 'Period ' + period[0] + ' to ' + period[1] + ' written in file.'

Next max_id: 670913730473955328
Remaining quota: {'reset': None, 'limit': None, 'remaining': 172}
Next max_id: 670772401794252800
Remaining quota: {'reset': None, 'limit': None, 'remaining': 171}
Next max_id: 670755256553349121
Remaining quota: {'reset': None, 'limit': None, 'remaining': 170}
Period 2015-11-29 to 2015-11-30 gathered.
Period 2015-11-29 to 2015-11-30 written in file.


In [145]:
def compute_feeling_avg(tweets_dico_list):
    texts_list = []
    for tweet in tweets_dico_list:
        texts_list.append(tweet['text'])
    print 'list finished'
    return senti_classifier.polarity_scores(texts_list)

In [146]:
print compute_feeling_avg(gather_tweets('%24GOOG', period[0], period[1]))

Next max_id: 670913730473955328
Remaining quota: {'reset': None, 'limit': None, 'remaining': 178}
Next max_id: 670772401794252800
Remaining quota: {'reset': None, 'limit': None, 'remaining': 177}
Next max_id: 670755256553349121
Remaining quota: {'reset': None, 'limit': None, 'remaining': 176}
list finished
(675.75, 418.75)
